# Init

In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [2]:
from __future__ import print_function
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, LSTM, GRU  #, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.optimizers import *
from keras.engine.topology import Layer
from keras.layers import *
from keras.utils import np_utils

import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [3]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [4]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [5]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [6]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [7]:
UID = "user_id"

# Load data (Only once)

In [8]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [9]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [10]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [11]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [12]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [13]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]

In [14]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [15]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [16]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Data Exploration (todo)

In [ ]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [ ]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")
# pd.DataFrame(np.sort(test_click_log[UID].unique()), columns=[UID]).to_csv(f"{DDIR}/test/user.csv", index=False)
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
train_click_log.shape

In [ ]:
train_ad.shape

In [ ]:
train_user.shape

In [ ]:
test_click_log.shape

In [ ]:
test_ad.shape

In [ ]:
train_ad["product_id"] = train_ad["product_id"].replace("\\N", -1).astype(int)
train_ad["industry"] = train_ad["industry"].replace("\\N", -1).astype(int)

In [ ]:
test_ad["product_id"] = test_ad["product_id"].replace("\\N", -1).astype(int)
test_ad["industry"] = test_ad["industry"].replace("\\N", -1).astype(int)

In [ ]:
# creative id in train (creative id is unique in train_ad)
len(train_ad)

In [ ]:
# creative id in test (creative id is unique in test_ad)
len(test_ad)

In [ ]:
# check whether the same creative_id in train and test have same ad info
insect1d = np.intersect1d(train_click_log.creative_id.unique(), test_click_log.creative_id.unique())
print("Same creative id: ", insect1d.shape)
print("Diff number: ", np.sum(train_ad[train_ad.creative_id.isin(insect1d)].values != test_ad[test_ad.creative_id.isin(insect1d)].values))
# checked: they all have same ad info (result is 0)

In [ ]:
# check whether click and ad have diff creative_id
print("Diff list: ", np.setdiff1d(train_click_log.creative_id.unique(), train_ad.creative_id))
print("Diff list: ", np.setdiff1d(train_ad.creative_id, train_click_log.creative_id.unique()))

In [ ]:
# check whether click and ad have diff creative_id
print("Diff list: ", np.setdiff1d(test_click_log.creative_id.unique(), test_ad.creative_id))
print("Diff list: ", np.setdiff1d(test_ad.creative_id, test_click_log.creative_id.unique()))

In [ ]:
# click time
sns.lineplot(x=train_click_log.time.value_counts().index, y=train_click_log.time.value_counts())

In [ ]:
sns.lineplot(x=test_click_log.time.value_counts().index, y=test_click_log.time.value_counts())

In [ ]:
# data_grouped = data.groupby(data.index)
# results = Parallel(n_jobs=8)(delayed(key_func)(group) for name, group in data_grouped)
# data = pd.concat(results)

# Feature engineering

## Get ID sequence (Only once)

In [ ]:
# # sort by time, for time series
# train_click_log.sort_values(by="time", inplace=True)
# test_click_log.sort_values(by="time", inplace=True)

In [ ]:
# tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
# tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [ ]:
# del train_click_log, test_click_log
# gc.collect()

In [ ]:
# @timeit
# def gen_id_series(data, dtyp="train"):
#     for col in ID_SET:
#         tmp = data.groupby([UID], sort=False)[[col]].agg(lambda x: [f"word_{y}" for y in x])
#         tmp.columns = bch_rencol(tmp.columns)
#         tmp.to_pickle(f"{UDDIR}/imd/{dtyp}_{col}_seq.pkl")
#         tmp = None

In [ ]:
# gen_id_series(tol_train, "train")

In [ ]:
# gen_id_series(tol_test, "test")

In [ ]:
# seqidx_dic = dict()

In [ ]:
# for col in ID_SET:
#     train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
#     test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
#     tol_seq = pd.concat([train_seq, test_seq])
#     seqidx_dic[col] = tol_seq.index

In [ ]:
# with open(f"{UDDIR}/imd/seqidx_dic.pkl", "wb") as f:
#     pickle.dump(pickle.dumps(seqidx_dic), f)

## TF-IDF&Count

In [ ]:
@timeit
def gen_tfidf(col, nr_max=1):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    tol_seq[col] = tol_seq[col].apply(lambda x: " ".join(x))
    
    train_seq = None
    test_seq = None
    tfidf_enc = TfidfVectorizer(ngram_range=(1, nr_max), min_df=30)
    tfidf_vec = tfidf_enc.fit_transform(tol_seq[col].values)
    log(f"TF-IDF shape: {tfidf_vec.shape}")

    # save sparse matrix
    scipy.sparse.save_npz(f"{UDDIR}/imd/sparse_tfidf_{col}.npz", tfidf_vec)

In [ ]:
@timeit
def gen_count(col, nr_max=1):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    tol_seq[col] = tol_seq[col].apply(lambda x: " ".join(x))
    
    train_seq = None
    test_seq = None
    count_enc = CountVectorizer(ngram_range=(1, nr_max), min_df=30)
    count_vec = count_enc.fit_transform(tol_seq[col].values)
    log(f"Count shape: {count_vec.shape}")

    # save sparse matrix
    scipy.sparse.save_npz(f"{UDDIR}/imd/sparse_count_{col}.npz", count_vec)

In [ ]:
for col in ID_SET[-2:]:
    gen_tfidf(col)

In [ ]:
for col in ID_SET[-2:]:
    gen_count(col)

### SVD

In [ ]:
@timeit
def gen_svd(col, index, prefix="tfidf", n_cpt=64):
    tfidf_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_{prefix}_{col}.npz")
    if tfidf_vec.shape[1] > n_cpt:
        svd_enc = TruncatedSVD(n_components=n_cpt, n_iter=20, random_state=2020)
        mode_svd = svd_enc.fit_transform(tfidf_vec)
    else:
        n_cpt = tfidf_vec.shape[1]
        mode_svd = tfidf_vec.todense()
    mode_svd = pd.DataFrame(mode_svd)
    mode_svd.columns = [f"{prefix}_svd_{col}_{i}" for i in range(n_cpt)]
    mode_svd.index = index
    # save svd pkl
    mode_svd.to_pickle(f"{UFEDIR}/{prefix}_svd_{col}.pkl")

In [ ]:
with open(f"{UDDIR}/imd/seqidx_dic.pkl", "rb") as f:
    seqidx_dic = pickle.loads(pickle.load(f))

In [ ]:
for col in ID_SET:
    gen_svd(col, seqidx_dic[col])

### Stacking

In [ ]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
sparse_matrix = None
for col in tqdm(ID_SET):
    print("current filename: ", f"{UDDIR}/imd/sparse_tfidf_{col}.npz", f"{UDDIR}/imd/sparse_count_{col}.npz")
    tfidf_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_tfidf_{col}.npz")
    count_vec = scipy.sparse.load_npz(f"{UDDIR}/imd/sparse_count_{col}.npz")
    sparse_matrix = scipy.sparse.csr_matrix(scipy.sparse.hstack([sparse_matrix, tfidf_vec]))
    sparse_matrix = scipy.sparse.csr_matrix(scipy.sparse.hstack([sparse_matrix, count_vec]))

In [ ]:
sparse_matrix.shape

In [ ]:
with open(f"{UDDIR}/imd/seqidx_dic.pkl", "rb") as f:
    seqidx_dic = pickle.loads(pickle.load(f))

In [ ]:
comb_ids = list(combinations(ID_SET, 2))

In [ ]:
is_eq = True
for cp in comb_ids:
    if (seqidx_dic[cp[0]] != seqidx_dic[cp[1]]).sum() != 0:
        print(cp)
        is_eq = False
        break
assert is_eq, "Must True"

In [ ]:
X_train = sparse_matrix[seqidx_dic["creative_id"].isin(train_user[UID])]
X_test = sparse_matrix[~seqidx_dic["creative_id"].isin(train_user[UID])]

In [ ]:
# reorder index
re_train_user = pd.DataFrame(index=seqidx_dic["creative_id"][seqidx_dic["creative_id"].isin(train_user[UID])])
re_train_user = pd.merge(re_train_user, train_user, how="left", on=UID)

In [ ]:
# reorder index
re_test_user = pd.DataFrame(index=seqidx_dic["creative_id"][~seqidx_dic["creative_id"].isin(train_user[UID])])
re_test_user = pd.merge(re_test_user, test_user, how="left", on=UID)

In [ ]:
log(X_train.shape)
log(X_test.shape)

In [ ]:
del sparse_matrix

In [ ]:
gc.collect()

#### For age

In [ ]:
tm_clf = Timer()

In [ ]:
random_seed = 2020
num_classes = 10
n_splits = 5
y = re_train_user["age"]
model_zoo = [SGDClassifier(n_jobs=-1,verbose=1), SGDClassifier(loss='log',n_jobs=-1,verbose=1),
             SGDClassifier(loss='modified_huber',n_jobs=-1,verbose=1),
             PassiveAggressiveClassifier(n_jobs=-1,verbose=1),LogisticRegression(C=10),
             RidgeClassifier(solver='lsqr',fit_intercept=False),LinearSVC(verbose=1,max_iter=500),
             BernoulliNB(),MultinomialNB()]
oof = []
count = 0
columns = ['SGD_HINGE', 'SGD_LOG','SGD_HUBER','PAC','LR','RIDGE','LSVC','BNB','MNB']

for model in model_zoo:
    cv_pred_stack = np.zeros((X_train.shape[0],num_classes))
    test_pred_stack = np.zeros((X_test.shape[0],num_classes))
    skf = KFold(n_splits=n_splits,random_state=random_seed)
    
    gc.collect()
    
    if os.path.exists(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl"):
        tmp = pd.read_pickle(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl")
    else:
        for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
            print(index,model)
            
            train_x = test_x = train_y = test_y = None
            gc.collect()
            
            train_x, test_x, train_y, test_y = X_train[train_index], X_train[test_index], y.iloc[train_index], y.iloc[test_index]
            model.fit(train_x,train_y)
            try:
                y_val = model._predict_proba_lr(test_x)
            except:
                y_val = model.predict_proba(test_x)
            cv_pred_stack[test_index] = y_val
            print(y_val.shape)
            try:
                test_pred_stack += model._predict_proba_lr(X_test) / n_splits
            except:
                test_pred_stack += model.predict_proba(X_test) / n_splits
            
                
        print(model,'score:',accuracy_score(y,np.argmax(cv_pred_stack,axis=1)))

        a = pd.DataFrame(cv_pred_stack).add_prefix(columns[count]+"_")
        a[UID] = re_train_user[UID].values
        b = pd.DataFrame(test_pred_stack).add_prefix(columns[count]+"_")
        b[UID] = re_test_user[UID].values
        tmp = a.append(b)
        tmp.to_pickle(f"{UDDIR}/imd/tfidf_count_emb_age_{columns[count]}.pkl")
        
    count += 1
    oof.append(tmp)
    

df_agg = pd.DataFrame()
for i in tqdm(oof):
    df_agg[i.columns] = i
df_agg.to_pickle(f"{UFEDIR}/tfidf_count_emb_age_all.pkl")

In [ ]:
tm_clf.check("clf embedding")

#### For gender

In [ ]:
random_seed = 2020
num_classes = 2
n_splits = 5
y = re_train_user["gender"]
model_zoo = [SGDClassifier(n_jobs=-1,verbose=1), SGDClassifier(loss='log',n_jobs=-1,verbose=1),
             SGDClassifier(loss='modified_huber',n_jobs=-1,verbose=1),
             PassiveAggressiveClassifier(n_jobs=-1,verbose=1),LogisticRegression(C=10),
             RidgeClassifier(solver='lsqr',fit_intercept=False),LinearSVC(verbose=1,max_iter=500),
             BernoulliNB(),MultinomialNB()]
oof = []
count = 0
columns = ['SGD_HINGE', 'SGD_LOG','SGD_HUBER','PAC','LR','RIDGE','LSVC','BNB','MNB']

for model in model_zoo:
    cv_pred_stack = np.zeros((X_train.shape[0],num_classes))
    test_pred_stack = np.zeros((X_test.shape[0],num_classes))
    skf = KFold(n_splits=n_splits,random_state=random_seed)
    
    gc.collect()
    
    if os.path.exists(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl"):
        tmp = pd.read_pickle(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl")
    else:
        for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
            print(index,model)

            train_x = test_x = train_y = test_y = None
            gc.collect()

            train_x, test_x, train_y, test_y = X_train[train_index], X_train[test_index], y.iloc[train_index], y.iloc[test_index]
            model.fit(train_x,train_y)
            try:
                y_val = model._predict_proba_lr(test_x)
            except:
                y_val = model.predict_proba(test_x)
            cv_pred_stack[test_index] = y_val
            print(y_val.shape)
            try:
                test_pred_stack += model._predict_proba_lr(X_test) / n_splits
            except:
                test_pred_stack += model.predict_proba(X_test) / n_splits
        print(model,'score:',accuracy_score(y,np.argmax(cv_pred_stack,axis=1)))

        a = pd.DataFrame(cv_pred_stack).add_prefix(columns[count]+"_")
        a[UID] = re_train_user[UID].values
        b = pd.DataFrame(test_pred_stack).add_prefix(columns[count]+"_")
        b[UID] = re_test_user[UID].values
        tmp = a.append(b)
        tmp.to_pickle(f"{UDDIR}/imd/tfidf_count_emb_gender_{columns[count]}.pkl")
        
    count += 1
    oof.append(tmp)
    

df_agg = pd.DataFrame()
for i in tqdm(oof):
    df_agg[i.columns] = i
df_agg.to_pickle(f"{UFEDIR}/tfidf_count_emb_gender_all.pkl")

In [ ]:
tm_clf.check("clf embedding")

## Word2Vec

In [ ]:
@timeit
def gen_w2v(col, vesize=300, win=20):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    if os.path.exists(f"{UMDIR}/vectors/w2v_{col}.model"):
        model = gensim.models.Word2Vec.load(f"{UMDIR}/vectors/w2v_{col}.model")
    else:
        model = gensim.models.Word2Vec(sentences=tol_seq[col], size=vesize, window=win, workers=32, sg=0, iter=10)
        model.save(f"{UMDIR}/vectors/w2v_{col}.model")
    
    w2v_list = list()
    
    for it in tqdm(tol_seq[col]):
        tmp = np.zeros(vesize)
        cnt = 0
        for wd in it:
            cnt += 1
            if wd in model:
                tmp += model[wd]
        w2v_list.append(list(tmp/cnt))
    
    w2v_avg = pd.DataFrame(w2v_list)
    w2v_avg.index = tol_seq.index
    w2v_avg.columns = [f"w2v_avg_{col}_{i}" for i in range(vesize)]
    w2v_avg = reduce_mem_usage(w2v_avg)
    w2v_avg.to_pickle(f"{UFEDIR}/w2v_avg_{col}.pkl")

In [ ]:
for col in ID_SET:
    gen_w2v(col)

## Doc2Vec

In [ ]:
@timeit
def gen_d2v(col, vesize=300, win=20):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    
    if os.path.exists(f"{UMDIR}/vectors/d2v_{col}.model"):
        model = gensim.models.Doc2Vec.load(f"{UMDIR}/vectors/d2v_{col}.model")
    else:
        docs = [gensim.models.doc2vec.TaggedDocument(words=i[1],tags=[str(i[0])]) for i in tol_seq[col].reset_index().values]
        model = gensim.models.Doc2Vec(documents=docs, size=vesize, window=win, workers=32, iter=10)
        model.save(f"{UMDIR}/vectors/d2v_{col}.model")
    
    d2v_list = list()
    for it, cps in tqdm(tol_seq[col].reset_index().values):
#         if it in model.docvecs:
        d2v_list.append(model.docvecs[str(it)])
#         else:
#             d2v_list.append(model.infer_vector(cps))

    d2v_avg = pd.DataFrame(d2v_list)
    d2v_avg.index = tol_seq.index
    d2v_avg.columns = [f"d2v_avg_{col}_{i}" for i in range(vesize)]
    d2v_avg = reduce_mem_usage(d2v_avg)
    d2v_avg.to_pickle(f"{UFEDIR}/d2v_avg_{col}.pkl")

In [ ]:
for col in ID_SET:
    gen_d2v(col)

## Glove

In [ ]:
@timeit
def gen_glove(col, vesize=300, win=20):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    tol_seq = pd.concat([train_seq, test_seq])
    
    train_seq = None
    test_seq = None
    
    if os.path.exists(f"{UMDIR}/vectors/glove_{col}.model"):
        glove = Glove.load(f"{UMDIR}/vectors/glove_{col}.model")
    else:
        matrix =  Corpus()
        matrix.fit(tol_seq[col].values)
        glove = Glove(no_components=vesize, learning_rate=0.05)
        glove.fit(matrix.matrix,epochs=10,no_threads=30,verbose=1)
        glove.add_dictionary(matrix.dictionary)
        glove.save(f"{UMDIR}/vectors/glove_{col}.model")
        
    ans = []
    for i in tqdm(tol_seq[col].values):
        line = []
        for j in i:
            line.append(glove.word_vectors[glove.dictionary[j]])
        ans.append(np.mean(line,axis=0))
    
    glove_avg = pd.DataFrame(ans)
    glove_avg.index = tol_seq.index
    glove_avg.columns = [f"glove_avg_{col}_{i}" for i in range(vesize)]
    glove_avg = reduce_mem_usage(glove_avg)
    glove_avg.to_pickle(f"{UFEDIR}/glove_avg_{col}.pkl")

In [ ]:
for col in ID_SET:
    gen_glove(col)

## Fasttext (TODO)

## Stats features

In [ ]:
tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [ ]:
del train_click_log, train_ad
del test_click_log, test_ad

### Category map to Label (use KFold TODO)

__a little overfitting age__

In [ ]:
@timeit
def gen_catemlb(train_data, test_data, tag, col):
    train_use = pd.merge(train_data[[UID, col]], tag, how="left", on=UID)
    test_use = test_data[[UID, col]]
    
    nfolds = 5
    kfold = KFold(n_splits=nfolds)
    
    kf_map_df = pd.DataFrame()

    re_train_use = pd.DataFrame()
    re_test_use = pd.DataFrame()

    for tr_idx, val_idx in kfold.split(train_use):
        tr_ucid, val_ucid = train_use.iloc[tr_idx], train_use.iloc[val_idx]
        kf_map = tr_ucid.groupby([col])[["age", "gender"]].agg(["mean"])
        kf_map.columns = bch_rencol(kf_map.columns, prefix=f"{col}_")
        # only use intersect between train and test
        kf_map.drop(np.setdiff1d(kf_map.index.unique(), test_use[col].unique()), inplace=True)
        # get kf_cum
        if kf_map_df.empty:
            kf_map_df = kf_map
        else:
            kf_map_df = (kf_map_df + kf_map).fillna(0)
        
        val_ucid = pd.merge(val_ucid, kf_map, how="left", on=col)
        re_train_use = pd.concat([re_train_use, val_ucid])
    
    kf_map_df = kf_map_df/nfolds
    re_test_use = pd.merge(test_use, kf_map_df, how="left", on=col)

    assert len(train_use) == len(re_train_use)
    assert len(test_use) == len(re_test_use)

    train_use = None
    test_use = None

    tmp = re_train_use.groupby([UID], sort=False)[f"{col}_age_mean", f"{col}_gender_mean"].agg(["sum", "max", "mean", "min", "std"])
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/train_stats_catemlb_{col}.pkl")
    tmp = None

    tmp = re_test_use.groupby([UID], sort=False)[f"{col}_age_mean", f"{col}_gender_mean"].agg(["sum", "max", "mean", "min", "std"])
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/test_stats_catemlb_{col}.pkl")
    tmp = None
    

In [ ]:
for col in ["creative_id",]:  # "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    gen_catemlb(tol_train, tol_test, train_user, col)

### One key(O1)-Time cost

In [ ]:
# for train
tmp = tol_train.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": OP_SET1,
        "time": OP_SET1,
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

tmp.columns = bch_rencol(tmp.columns)

tmp.to_pickle(f"{UFEDIR}/train_stats_o1.pkl")
tmp = None

In [ ]:
# for test
tmp = tol_test.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": OP_SET1,
        "time": OP_SET1,
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

tmp.columns = bch_rencol(tmp.columns)

tmp.to_pickle(f"{UFEDIR}/test_stats_o1.pkl")
tmp = None

In [ ]:
gc.collect()

### Comb Key(O2)-Time cost

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for i in tqdm(range(1, 7)):
    for cl in combinations(id_list, i):
        key_set = [UID] + list(cl)
        tmp = tol_train.groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"{'C'.join(key_set[1:-1])}_")
        tmp.to_pickle(f"{UFEDIR}/train_stats_o2_{'C'.join(key_set)}.pkl")
        tmp = None

In [ ]:
# test
for i in tqdm(range(1, 7)):
    for cl in combinations(id_list, i):
        key_set = [UID] + list(cl)
        tmp = tol_test.groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"{'C'.join(key_set[1:-1])}_")
        tmp.to_pickle(f"{UFEDIR}/test_stats_o2_{'C'.join(key_set)}.pkl")
        tmp = None

In [ ]:
gc.collect()

### Time Windows-Time cost

In [ ]:
tm_tw = Timer()

#### Time Bins

In [ ]:
bins_set = [3, 7, 10, 30]

for bins in bins_set:
    tol_bins = pd.cut(pd.concat([tol_train["time"], tol_test["time"]]), bins, labels=range(bins))
    tol_train[f"bins{bins}"] = tol_bins[:len(tol_train)]
    tol_test[f"bins{bins}"] = tol_bins[len(tol_train):]

In [ ]:
tm_tw.check("Init Bin Set")

##### One Key(O1)

In [ ]:
# For train
for bins in bins_set:
    for cb in tqdm(range(bins)):
        tmp = tol_train[tol_train[f"bins{bins}"] == cb].groupby([UID], sort=False).agg(
            {
                UID: ["count"], 
                "click_times": OP_SET1,
                "time": OP_SET1,
                "creative_id": ["nunique"],
                "ad_id": ["nunique"],
                "product_id": ["nunique"],
                "product_category": ["nunique"],
                "advertiser_id": ["nunique"],
                "industry": ["nunique"],
            }
        )
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_")
        tmp.to_pickle(f"{UFEDIR}/train_stats_o1_bins{bins}_{cb}.pkl")
        tmp = None

In [ ]:
tm_tw.check("Generate o1 for train")

In [ ]:
# For test
for bins in bins_set:
    for cb in tqdm(range(bins)):
        tmp = tol_test[tol_test[f"bins{bins}"] == cb].groupby([UID], sort=False).agg(
            {
                UID: ["count"], 
                "click_times": OP_SET1,
                "time": OP_SET1,
                "creative_id": ["nunique"],
                "ad_id": ["nunique"],
                "product_id": ["nunique"],
                "product_category": ["nunique"],
                "advertiser_id": ["nunique"],
                "industry": ["nunique"],
            }
        )
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_")
        tmp.to_pickle(f"{UFEDIR}/test_stats_o1_bins{bins}_{cb}.pkl")
        tmp = None

In [ ]:
tm_tw.check("Generate o1 for test")

In [ ]:
gc.collect()

##### Comb Key(O2)

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for bins in bins_set:
    for cb in tqdm(range(bins)):
        for i in tqdm(range(1, 7)):
            for cl in combinations(id_list, i):
                key_set = [UID] + list(cl)
                tmp = tol_train[tol_train[f"bins{bins}"] == cb].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
                tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_{'C'.join(key_set[1:-1])}_")
                tmp.to_pickle(f"{UFEDIR}/train_stats_o2_bins{bins}_{cb}_{'C'.join(key_set)}.pkl")
                tmp = None

In [ ]:
tm_tw.check("Generate o2 bin for train")

In [ ]:
# test
for bins in bins_set:
    for cb in tqdm(range(bins)):
        for i in tqdm(range(1, 7)):
            for cl in combinations(id_list, i):
                key_set = [UID] + list(cl)
                tmp = tol_test[tol_test[f"bins{bins}"] == cb].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
                tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{cb}_{'C'.join(key_set[1:-1])}_")
                tmp.to_pickle(f"{UFEDIR}/test_stats_o2_bins{bins}_{cb}_{'C'.join(key_set)}.pkl")
                tmp = None

In [ ]:
tm_tw.check("Generate o2 bin for test")

#### Time Slides

In [ ]:
slide_set = [1, 2, 3, 7, 14, 21, 30, 60, 90]

tol_train["max_time"] = tol_train.groupby([UID], sort=False)["time"].transform("max")
tol_test["max_time"] = tol_test.groupby([UID], sort=False)["time"].transform("max")

In [ ]:
tm_tw.check("Get Max time")

##### One Key(O1)

In [ ]:
# For train
for win in slide_set:
    tmp = tol_train[tol_train["time"] >= tol_train["max_time"] - win].groupby([UID], sort=False).agg(
        {
            UID: ["count"], 
            "click_times": OP_SET1,
            "time": OP_SET1,
            "creative_id": ["nunique"],
            "ad_id": ["nunique"],
            "product_id": ["nunique"],
            "product_category": ["nunique"],
            "advertiser_id": ["nunique"],
            "industry": ["nunique"],
        }
    )
    tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_")
    tmp.to_pickle(f"{UFEDIR}/train_stats_o1_slide{win}.pkl")
    tmp = None

In [ ]:
tm_tw.check("Generate o1 slide for train")

In [ ]:
# For test
for win in slide_set:
    tmp = tol_test[tol_test["time"] >= tol_test["max_time"] - win].groupby([UID], sort=False).agg(
        {
            UID: ["count"], 
            "click_times": OP_SET1,
            "time": OP_SET1,
            "creative_id": ["nunique"],
            "ad_id": ["nunique"],
            "product_id": ["nunique"],
            "product_category": ["nunique"],
            "advertiser_id": ["nunique"],
            "industry": ["nunique"],
        }
    )
    tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_")
    tmp.to_pickle(f"{UFEDIR}/test_stats_o1_slide{win}.pkl")
    tmp = None

In [ ]:
tm_tw.check("Generate o1 slide for test")

##### Comb Key(O2)

In [ ]:
id_list = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
# train
for win in slide_set:
    for i in tqdm(range(1, 7)):
        for cl in combinations(id_list, i):
            key_set = [UID] + list(cl)
            tmp = tol_train[tol_train["time"] >= tol_train["max_time"] - win].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
            tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_{'C'.join(key_set[1:-1])}_")
            tmp.to_pickle(f"{UFEDIR}/train_stats_o2_slide{win}_{'C'.join(key_set)}.pkl")
            tmp = None

In [ ]:
tm_tw.check("Generate o2 slide for train")

In [ ]:
# test
for win in slide_set:
    for i in tqdm(range(1, 7)):
        for cl in combinations(id_list, i):
            key_set = [UID] + list(cl)
            tmp = tol_test[tol_test["time"] >= tol_test["max_time"] - win].groupby(key_set, sort=False)[key_set[-1:]].agg(["count"]).groupby([UID]).agg(OP_SET2)
            tmp.columns = bch_rencol(tmp.columns, prefix=f"slide{win}_{'C'.join(key_set[1:-1])}_")
            tmp.to_pickle(f"{UFEDIR}/test_stats_o2_slide{win}_{'C'.join(key_set)}.pkl")
            tmp = None

In [ ]:
tm_tw.check("Generate o2 slide for test")

In [ ]:
gc.collect()

### (TOP Category) One-Hot

In [ ]:
tol_data = pd.concat([tol_train, tol_test])

In [ ]:
dum_dic = {
    "creative_id": 300,
    "ad_id": 300,
    "product_id": 100,
    "advertiser_id": 100,
}

In [ ]:
for cid in dum_dic:
    use_id = tol_data[cid].value_counts().head(dum_dic[cid]).index.values
    tol_data[f"{cid}_dummy"] = tol_data[cid]
    tol_data[f"{cid}_dummy"][~tol_data[f"{cid}_dummy"].isin(use_id)] = -999

In [ ]:
for cid in  ["time", "click_times", "creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    ocid = cid
    if cid in dum_dic:
        cid = f"{cid}_dummy"
    tmp = tol_data[[UID, cid]].groupby([UID, cid], sort=False)[[cid]].agg(["count"]).unstack().fillna(0)
    tmp.columns = bch_rencol(tmp.columns)
    reduce_mem_usage(tmp).to_pickle(f"{UFEDIR}/onehot_catecnt_{ocid}.pkl")
    tmp = None

In [ ]:
gc.collect()

## Meta Train 

__weak learner prediction as features__

In [ ]:
#================Once=================#

In [ ]:
tol_test.index = -tol_test.index -1

In [ ]:
tol_data = pd.concat([tol_train, tol_test])

In [ ]:
del tol_train, tol_test
gc.collect()

In [ ]:
tol_data[f"{UID}_count"] = tol_data[[UID]].groupby([UID])[UID].transform("count")

In [ ]:
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    tmp = tol_data[[UID, col]]
    tol_data[f"{UID}_{col}_std"] = tmp.groupby([UID])[col].transform("std")
    tol_data[f"{col}_{UID}_count"] = tmp.groupby([col])[UID].transform("count")
    tol_data[f"{col}_{UID}_std"] = tmp.groupby([col])[UID].transform("std")
    tmp = None

In [ ]:
for col in ["time", "click_times"]:
    tmp = tol_data[[UID, col]]
    tol_data[f"{UID}_{col}_std"] = tmp.groupby([UID])[col].transform("std")
    tol_data[f"{UID}_{col}_mean"] = tmp.groupby([UID])[col].transform("mean")
    tmp = None

In [ ]:
tol_data = reduce_mem_usage(tol_data)

In [ ]:
tol_data.to_pickle(f"{UDDIR}/imd/tol_data.pkl")

In [ ]:
#==============End===============#

In [ ]:
## if have tol_data.pkl
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

tol_data = pd.read_pickle(f"{UDDIR}/imd/tol_data.pkl")

In [ ]:
def rmse(y_true, y_pred):
    return (mean_squared_error(y_true, y_pred))** .5

In [ ]:
gc.collect()

### For age

In [ ]:
tol_data = pd.merge(tol_data, train_user[[UID, "age"]], how="left", on=UID)

In [ ]:
# use df_hist_train df_new_train df_hist_new_train to train 3 models
train_df = tol_data[tol_data["age"].notnull()]
test_df = tol_data[tol_data["age"].isnull()]

In [ ]:
drop_features = [UID, "age"]
cat_features = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
feats = [f for f in tol_data.columns if f not in drop_features]

In [ ]:
n_splits= 3
folds = GroupKFold(n_splits=n_splits)
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
print ('feats:' + str(len(feats)))

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df["age"],groups=train_df[UID])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df["age"].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df["age"].iloc[valid_idx] 
    
    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
               "objective" : "regression", 
               "boosting" : "gbdt", 
               "metric" : "rmse",  
               "max_depth": 7, 
               "num_leaves" : 31, 
               "max_bin" : 255, 
               "learning_rate" : 0.1, 
               "subsample" : 0.8,
               "colsample_bytree" : 0.8, 
               "verbosity": -1,
               "num_threads" : -1,
    }
    

    if n_fold >= 0:
        evals_result = {}
        dtrain = lgb.Dataset(
            train_x, label=train_y,categorical_feature=cat_features)
        dval = lgb.Dataset(
            valid_x, label=valid_y, reference=dtrain,categorical_feature=cat_features)
        bst = lgb.train(
            params, dtrain, num_boost_round=30000,
            valid_sets=[dval], early_stopping_rounds=100, verbose_eval=20,)#feval = evalerror
        
        new_list = sorted(zip(feats, bst.feature_importance('gain')),key=lambda x: x[1], reverse=True)[:]
        for item in new_list:
            print (item) 

        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)

        sub_preds += bst.predict(test_df[feats], num_iteration=bst.best_iteration) / folds.n_splits # test_df_new

In [ ]:
cv = rmse(train_df["age"],  oof_preds)
print('Full OOF RMSE %.6f' % cv)  

In [ ]:
a = train_df[[UID]]
b = test_df[[UID]]

a["age_pred"] = oof_preds
b["age_pred"] = sub_preds

a1 = a.groupby([UID])["age_pred"].agg(["mean", "std", "min", "max", "median", "skew"])
b1 = b.groupby([UID])["age_pred"].agg(["mean", "std", "min", "max", "median", "skew"])

In [ ]:
a1.append(b1).add_prefix("age_gkf_agg_pred_").reset_index().sort_values(by=[UID]).reset_index(drop=True).to_pickle(f"{UFEDIR}/meta_age_group_regeress.pkl")

### For gender

In [ ]:
tol_data = pd.merge(tol_data, train_user[[UID, "gender"]], how="left", on=UID)

In [ ]:
# use df_hist_train df_new_train df_hist_new_train to train 3 models
train_df = tol_data[tol_data["gender"].notnull()]
test_df = tol_data[tol_data["gender"].isnull()]

In [ ]:
drop_features = [UID, "gender"]
cat_features = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
feats = [f for f in tol_data.columns if f not in drop_features]

In [ ]:
n_splits= 3
folds = GroupKFold(n_splits=n_splits)
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
print ('feats:' + str(len(feats)))

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df["gender"],groups=train_df[UID])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df["gender"].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df["gender"].iloc[valid_idx] 
    
    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
               "objective" : "regression", 
               "boosting" : "gbdt", 
               "metric" : "rmse",  
               "max_depth": 7, 
               "num_leaves" : 31, 
               "max_bin" : 255, 
               "learning_rate" : 0.1, 
               "subsample" : 0.8,
               "colsample_bytree" : 0.8, 
               "verbosity": -1,
               "num_threads" : -1,
    }
    

    if n_fold >= 0:
        evals_result = {}
        dtrain = lgb.Dataset(
            train_x, label=train_y,categorical_feature=cat_features)
        dval = lgb.Dataset(
            valid_x, label=valid_y, reference=dtrain,categorical_feature=cat_features)
        bst = lgb.train(
            params, dtrain, num_boost_round=30000,
            valid_sets=[dval], early_stopping_rounds=100, verbose_eval=20,)#feval = evalerror
        
        new_list = sorted(zip(feats, bst.feature_importance('gain')),key=lambda x: x[1], reverse=True)[:]
        for item in new_list:
            print (item) 

        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)

        sub_preds += bst.predict(test_df[feats], num_iteration=bst.best_iteration) / folds.n_splits # test_df_new

In [ ]:
cv = rmse(train_df["gender"],  oof_preds)
print('Full OOF RMSE %.6f' % cv)

In [ ]:
a = train_df[[UID]]
b = test_df[[UID]]

a["gender_pred"] = oof_preds
b["gender_pred"] = sub_preds

a1 = a.groupby([UID])["gender_pred"].agg(["mean", "std", "min", "max", "median", "skew"])
b1 = b.groupby([UID])["gender_pred"].agg(["mean", "std", "min", "max", "median", "skew"])

In [ ]:
a1.append(b1).add_prefix("gender_gkf_agg_pred_").reset_index().sort_values(by=[UID]).reset_index(drop=True).to_pickle(f"{UFEDIR}/meta_gender_group_regeress.pkl")

# Concat Feature

In [ ]:
feat_fname = sorted(os.listdir(UFEDIR))

In [ ]:
# # reduce memory (only once)
# for fname in feat_fname:
#     if fname.startswith("w2v_") or fname.startswith("tfidf_svd_") or fname.startswith("meta_age_") or fname.startswith("meta_gender_") or (fname.find("catemlb") != -1) or (fname.find("stats") != -1):
#         print("current filename: ", fname)
#         reduce_mem_usage(pd.read_pickle(f"{UFEDIR}/{fname}")).to_pickle(f"{UFEDIR}/{fname}")

In [ ]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()
train_feat[UID] = train_user[UID]
test_feat[UID] = test_user[UID]

In [ ]:
for fname in feat_fname:
    if fname.startswith("w2v_"):
        print("current filename: ", fname)
        cur_w2v = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, cur_w2v, how="left", on=UID)
        test_feat = pd.merge(test_feat, cur_w2v, how="left", on=UID)
        cur_w2v = None

In [ ]:
# stats o1
for fname in feat_fname:
    if fname.startswith("train_stats_o1"):
        print("current filename: ", fname)
        train_feat = pd.merge(train_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o1"):
        print("current filename: ", fname)
        test_feat = pd.merge(test_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# stats getidxmax
for fname in feat_fname:
    if fname.startswith("train_stats_o2_getidxmax"):
        print("current filename: ", fname)
        train_feat = pd.merge(train_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o2_getidxmax"):
        print("current filename: ", fname)
        test_feat = pd.merge(test_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# # stats getidxmax
# for fname in feat_fname:
#     if fname.startswith("train_stats_catemlb"):
#         print("current filename: ", fname)
#         train_feat = pd.merge(train_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
#     elif fname.startswith("test_stats_catemlb"):
#         print("current filename: ", fname)
#         test_feat = pd.merge(test_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [ ]:
# not use
# for fname in feat_fname:
#     if fname.startswith("tfidf_svd_"):
#         print("current filename: ", fname)
#         cur_tfidf_svd = pd.read_pickle(f"{UFEDIR}/{fname}")
#         train_feat = pd.merge(train_feat, cur_tfidf_svd, how="left", on=UID)
#         test_feat = pd.merge(test_feat, cur_tfidf_svd, how="left", on=UID)
#         cur_tfidf_svd = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("meta_age_"):
        print("current filename: ", fname)
        age_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, age_agg_pred, how="left", on=UID)
        test_feat = pd.merge(test_feat, age_agg_pred, how="left", on=UID)
        age_agg_pred = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("meta_gender_"):
        print("current filename: ", fname)
        gender_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, gender_agg_pred, how="left", on=UID)
        test_feat = pd.merge(test_feat, gender_agg_pred, how="left", on=UID)
        gender_agg_pred = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("d2v_"):
        print("current filename: ", fname)
        cur_d2v = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, cur_d2v, how="left", on=UID)
        test_feat = pd.merge(test_feat, cur_d2v, how="left", on=UID)
        cur_d2v = None

In [ ]:
for fname in feat_fname:
    if fname.startswith("glove_"):
        print("current filename: ", fname)
        cur_glove = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, cur_glove, how="left", on=UID)
        test_feat = pd.merge(test_feat, cur_glove, how="left", on=UID)
        cur_glove = None

In [ ]:
# # not use has same effect with w2v and doc2vec
# for fname in feat_fname:
#     if fname.startswith("onehot_catecnt_"):
#         print("current filename: ", fname)
#         cur_ohcc = pd.read_pickle(f"{UFEDIR}/{fname}")
#         train_feat = pd.merge(train_feat, cur_ohcc, how="left", on=UID)
#         test_feat = pd.merge(test_feat, cur_ohcc, how="left", on=UID)

In [ ]:
for fname in feat_fname:
    if fname.startswith("tfidf_count_emb_"):
        print("current filename: ", fname)
        cur_tce = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_feat = pd.merge(train_feat, cur_tce, how="left", on=UID)
        test_feat = pd.merge(test_feat, cur_tce, how="left", on=UID)

In [ ]:
# to make sure feat and user(target) have same order
# if true --> sum == 0
np.sum(train_feat[UID] != train_user[UID])

In [ ]:
# to make sure feat and user(target) have same order
# if true --> sum == 0
np.sum(test_feat[UID] != test_user[UID])

In [ ]:
# to make sure train and test features have same order
# if true --> sum == 0
np.sum(train_feat.columns != test_feat.columns)

In [ ]:
train_feat.shape

In [ ]:
test_feat.shape

In [ ]:
train_feat.memory_usage().sum() / 1024**2 

In [ ]:
test_feat.memory_usage().sum() / 1024**2 

In [ ]:
gc.collect()

In [ ]:
train_feat.to_pickle(f"{UDDIR}/feat_ing/train_feat_tol_v05.pkl")
test_feat.to_pickle(f"{UDDIR}/feat_ing/test_feat_tol_v05.pkl")

In [ ]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")
train_feat = pd.read_pickle(f"{UDDIR}/feat_ing/train_feat_tol_v05.pkl")
test_feat = pd.read_pickle(f"{UDDIR}/feat_ing/test_feat_tol_v05.pkl")

In [ ]:
# train_feat.drop([col for col in train_feat.columns if col.find("creative_id_gender_") != -1], axis=1, inplace=True)

In [ ]:
# test_feat.drop([col for col in test_feat.columns if col.find("creative_id_gender_") != -1], axis=1, inplace=True)

In [ ]:
# list(train_feat.columns)

# Training&Prediction

In [ ]:
train_feat.set_index(UID, inplace=True)
test_feat.set_index(UID, inplace=True)

In [ ]:
prob_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))

In [ ]:
gc.collect()

## NN

### Functions

In [17]:
class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())

In [18]:
class RAdam(Optimizer):

    """RAdam optimizer.

    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor. If `None`, defaults to `K.epsilon()`.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Weight decay for each param.
        amsgrad: boolean. Whether to apply the AMSGrad variant of this
            algorithm from the paper "On the Convergence of Adam and
            Beyond".
        total_steps: int >= 0. Total number of training steps. Enable warmup by setting a positive value.
        warmup_proportion: 0 < warmup_proportion < 1. The proportion of increasing steps.
        min_lr: float >= 0. Minimum learning rate after warmup.
    # References
        - [Adam - A Method for Stochastic Optimization](https://arxiv.org/abs/1412.6980v8)
        - [On the Convergence of Adam and Beyond](https://openreview.net/forum?id=ryQu7f-RZ)
        - [On The Variance Of The Adaptive Learning Rate And Beyond](https://arxiv.org/pdf/1908.03265v1.pdf)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, decay=0., weight_decay=0., amsgrad=False,
                 total_steps=0, warmup_proportion=0.1, min_lr=0., **kwargs):
        super(RAdam, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.weight_decay = K.variable(weight_decay, name='weight_decay')
            self.total_steps = K.variable(total_steps, name='total_steps')
            self.warmup_proportion = K.variable(warmup_proportion, name='warmup_proportion')
            self.min_lr = K.variable(min_lr, name='min_lr')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_decay = decay
        self.initial_weight_decay = weight_decay
        self.initial_total_steps = total_steps
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        lr = self.learning_rate

        if self.initial_decay > 0:
            lr = lr * (1. / (1. + self.decay * K.cast(self.iterations, K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1

        if self.initial_total_steps > 0:
            warmup_steps = self.total_steps * self.warmup_proportion
            decay_steps = self.total_steps - warmup_steps
            lr = K.switch(
                t <= warmup_steps,
                lr * (t / warmup_steps),
                lr * (1.0 - K.minimum(t, decay_steps) / decay_steps),
            )

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='m_' + str(i)) for (i, p) in enumerate(params)]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='v_' + str(i)) for (i, p) in enumerate(params)]

        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p), name='vhat_' + str(i)) for (i, p) in enumerate(params)]
        else:
            vhats = [K.zeros(1, name='vhat_' + str(i)) for i in range(len(params))]

        self.weights = [self.iterations] + ms + vs + vhats

        beta_1_t = K.pow(self.beta_1, t)
        beta_2_t = K.pow(self.beta_2, t)

        sma_inf = 2.0 / (1.0 - self.beta_2) - 1.0
        sma_t = sma_inf - 2.0 * t * beta_2_t / (1.0 - beta_2_t)

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)

            m_corr_t = m_t / (1.0 - beta_1_t)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                v_corr_t = K.sqrt(vhat_t / (1.0 - beta_2_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                v_corr_t = K.sqrt(v_t / (1.0 - beta_2_t) + self.epsilon)

            r_t = K.sqrt((sma_t - 4.0) / (sma_inf - 4.0) *
                         (sma_t - 2.0) / (sma_inf - 2.0) *
                         sma_inf / sma_t)

            p_t = K.switch(sma_t > 5, r_t * m_corr_t / v_corr_t, m_corr_t)

            if self.initial_weight_decay > 0:
                p_t += self.weight_decay * p

            p_t = p - lr * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'lr': float(K.get_value(self.learning_rate)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'decay': float(K.get_value(self.decay)),
            'weight_decay': float(K.get_value(self.weight_decay)),
            'epsilon': self.epsilon,
            'amsgrad': self.amsgrad,
            'total_steps': float(K.get_value(self.total_steps)),
            'warmup_proportion': float(K.get_value(self.warmup_proportion)),
            'min_lr': float(K.get_value(self.min_lr)),
        }
        base_config = super(RAdam, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [19]:
class Lookahead(object):
    """Add the [Lookahead Optimizer](https://arxiv.org/abs/1907.08610) functionality for [keras](https://keras.io/).

    """
    def __init__(self, k=5, alpha=0.5):
        self.k = k
        self.alpha = alpha
        self.count = 0

    def inject(self, model):
        """Inject the Lookahead algorithm for the given model.
        The following code is modified from keras's _make_train_function method.
        See: https://github.com/keras-team/keras/blob/master/keras/engine/training.py#L497
        """
        if not hasattr(model, 'train_function'):
            raise RuntimeError('You must compile your model before using it.')

        model._check_trainable_weights_consistency()

        if model.train_function is None:
            inputs = (model._feed_inputs +
                      model._feed_targets +
                      model._feed_sample_weights)
            if model._uses_dynamic_learning_phase():
                inputs += [K.learning_phase()]
            fast_params = model._collected_trainable_weights

            with K.name_scope('training'):
                with K.name_scope(model.optimizer.__class__.__name__):
                    training_updates = model.optimizer.get_updates(
                        params=fast_params,
                        loss=model.total_loss)
                    slow_params = [K.variable(p) for p in fast_params]
                fast_updates = (model.updates +
                                training_updates +
                                model.metrics_updates)

                slow_updates, copy_updates = [], []
                for p, q in zip(fast_params, slow_params):
                    slow_updates.append(K.update(q, q + self.alpha * (p - q)))
                    copy_updates.append(K.update(p, q))

                # Gets loss and metrics. Updates weights at each call.
                fast_train_function = K.function(
                    inputs,
                    [model.total_loss] + model.metrics_tensors,
                    updates=fast_updates,
                    name='fast_train_function',
                    **model._function_kwargs)

                def F(inputs):
                    self.count += 1
                    R = fast_train_function(inputs)
                    if self.count % self.k == 0:
                        K.batch_get_value(slow_updates)
                        K.batch_get_value(copy_updates)
                    return R
                
                model.train_function = F

In [20]:
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [21]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

In [22]:
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.learning_rate = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            # decoupled weight decay (2/4)
            self.wd = K.variable(weight_decay, name='weight_decay')
        self.epsilon = epsilon
        self.initial_decay = decay

#     @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd  # decoupled weight decay (3/4)

        lr = self.learning_rate
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            # decoupled weight decay (4/4)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.learning_rate)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [23]:
class Position_Embedding(Layer): 
    def __init__(self, size=None, mode='sum', **kwargs):        
        self.size = size         
        self.mode = mode       
        super(Position_Embedding, self).__init__(**kwargs) 

    def call(self, x): 
        if (self.size == None) or (self.mode == 'sum'):            
            self.size = int(x.shape[-1])        
            batch_size, seq_len = K.shape(x)[0], K.shape(x)[1]        
            position_j = 1. / K.pow(10000., \
                2 * K.arange(self.size / 2, dtype='float32') / self.size)        
            position_j = K.expand_dims(position_j, 0)        
            position_i = K.cumsum(K.ones_like(x[:, :, 0]), 1)-1     
            position_i = K.expand_dims(position_i, 2)        
            position_ij = K.dot(position_i, position_j)        
            position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2) 
            if self.mode == 'sum': 
                return position_ij + x 
            elif self.mode == 'concat': 
                return K.concatenate([position_ij, x], 2) 

    def compute_output_shape(self, input_shape): 
        if self.mode == 'sum': 
            return input_shape 
        elif self.mode == 'concat': 
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)

In [24]:
class FM(Layer): 
    def __init__(self, output_dim=30, activation="relu",**kwargs): 
        self.output_dim = output_dim 
        self.activate = activations.get(activation)
        super(FM, self).__init__(**kwargs) 
        
    def build(self, input_shape): 
        self.weight = self.add_weight(name='weight',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        self.bias = self.add_weight(name='bias',shape=(self.output_dim,),initializer='zeros',trainable=True) 
        self.kernel = self.add_weight(name='kernel',shape=(input_shape[1], self.output_dim),initializer='glorot_uniform',trainable=True) 
        super(FM, self).build(input_shape) 
        
    def call(self, x):
        feature = K.dot(x,self.weight) + self.bias
        a = K.pow(K.dot(x,self.kernel), 2)
        b = K.dot(x, K.pow(self.kernel, 2))
        cross = K.mean(a-b, 1, keepdims=True)*0.5
        cross = K.repeat_elements(K.reshape(cross, (-1, 1)), self.output_dim, axis=-1) 
        return self.activate(feature + cross) 
    
    def compute_output_shape(self, input_shape): 
        return (input_shape[0], self.output_dim)

In [25]:
def baseline_model(inshape, outshape):
    # create two models
    input1 = Input(shape=inshape)
    embed_1 = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(input1)
#     lstm_1 = LSTM(256)
    dense_1 = Dense(128, kernel_initializer='normal', activation='relu')(input1)
#     dense_1 = Dropout(0.5)(dense_1)
    dense_2 = Dense(64, kernel_initializer='normal', activation='relu')(dense_1)
#     dense_2 = Dropout(0.5)(dense_2)
    dense_3 = Dense(32, kernel_initializer='normal', activation='relu')(dense_2)
#     dense_3 = Dropout(0.5)(dense_3)
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(dense_3)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer="adadelta", metrics=['accuracy'])  #logloss
    return model

In [26]:
def DenseNet(inshape, outshape):
    # create two models
    input1  = Input(shape=inshape)

    dense_1 = Dense(256, kernel_initializer='normal', activation='relu')(input1)
    #dense_1 = Dropout(0.6)(dense_1)
    dense_2 = Dense(128, kernel_initializer='normal', activation='relu')(dense_1)
    #dense_2 = Dropout(0.6)(dense_2)
    dense_2_x  = concatenate([dense_1,dense_2])
    dense_3 = Dense(64, kernel_initializer='normal', activation='relu')(dense_2_x)
    #dense_3 = Dropout(0.6)(dense_3)
    dense_3_x  = concatenate([dense_1,dense_2,dense_3])
    dense_4 = Dense(7, kernel_initializer='normal', activation='relu')(dense_3_x)
    #dense_4 = Dropout(0.6)(dense_4)
    dense_4_x  = concatenate([dense_1,dense_2,dense_3,dense_4])
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(dense_4_x)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [27]:
def FMNet(inshape, outshape):
    # create two models
    input1  = Input(shape=inshape)
    #DNN_model_I
    dense_1 = Dense(100, kernel_initializer='normal', activation='tanh')(input1)
    dense_2 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_1)
    dense_3 = Dense(150, kernel_initializer='normal', activation='tanh')(dense_2)
    dense_4 = Dense(100, kernel_initializer='normal', activation='tanh')(dense_3)
    dense_5 = Dense(64, kernel_initializer='normal', activation='tanh')(dense_4)
    #FM_model_II
    FM_1    = FM(200)(input1)
    FM_2    = FM(64)(FM_1)

    x       = concatenate([dense_5,FM_2])
    x_tmp   = Dense(32,kernel_initializer='normal', activation='softmax')(x)
    out     = Dense(outshape,kernel_initializer='normal', activation='softmax')(x_tmp)
    # Compile model
    model = Model(inputs=input1, outputs = out)
    model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    #model.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])  #logloss
    return model

In [28]:
def loadnprec(col, max_feature=None, maxlen=None):
    train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
    test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")
    
    train_X = train_seq[col].values
    test_X = test_seq[col].values
    
    # Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    
    train_X = tokenizer.texts_to_sequences(train_X)
    test_X = tokenizer.texts_to_sequences(test_X)
    
    # Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    test_X = pad_sequences(test_X, maxlen=maxlen)
    
    return train_X, test_X, tokenizer.word_index

In [29]:
def load_glove(col, word_index, max_features):
    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/glove_{col}.model")
    k = Glove.load(EMBEDDING_FILE)
    
    embeddings_index = []
    for i in tqdm(k.dictionary.keys()):
        embeddings_index.append((i,k.word_vectors[k.dictionary[i]]))

    embeddings_index = dict(pd.DataFrame(embeddings_index).values)
    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 

In [30]:
def load_w2v(col, word_index, max_features):    
    EMBEDDING_FILE = (f"{UMDIR}/vectors/glove_{col}.model")
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    # word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

In [31]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1],),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init,
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     name='{}_b'.format(self.name),
                                     initializer='zero',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

### Base NN class

In [32]:
feat_fname = sorted(os.listdir(UFEDIR))

In [33]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [34]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()
train_feat[UID] = train_user[UID]
test_feat[UID] = test_user[UID]

In [35]:
train_feat.set_index(UID, inplace=True)
test_feat.set_index(UID, inplace=True)

In [36]:
# word2vec

In [37]:
w2v_dic = dict()
for cur_id in ID_SET:
    for fname in feat_fname:
        if fname.startswith(f"w2v_avg_{cur_id}"):
            print("current filename: ", fname)
            w2v_dic[cur_id] = pd.read_pickle(f"{UFEDIR}/{fname}")

current filename:  w2v_avg_creative_id.pkl
current filename:  w2v_avg_ad_id.pkl
current filename:  w2v_avg_product_id.pkl
current filename:  w2v_avg_product_category.pkl
current filename:  w2v_avg_advertiser_id.pkl
current filename:  w2v_avg_industry.pkl


In [38]:
train_ctid = pd.merge(train_feat, w2v_dic["creative_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_adid = pd.merge(train_feat, w2v_dic["ad_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_atid = pd.merge(train_feat, w2v_dic["advertiser_id"], how="left", on=UID).values.reshape(-1, 1, 300)

train_pdid = pd.merge(train_feat, w2v_dic["product_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_pdct = pd.merge(train_feat, w2v_dic["product_category"], how="left", on=UID).values.reshape(-1, 1, 300)
train_inds = pd.merge(train_feat, w2v_dic["industry"], how="left", on=UID).values.reshape(-1, 1, 300)

In [39]:
test_ctid = pd.merge(test_feat, w2v_dic["creative_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_adid = pd.merge(test_feat, w2v_dic["ad_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_atid = pd.merge(test_feat, w2v_dic["advertiser_id"], how="left", on=UID).values.reshape(-1, 1, 300)

test_pdid = pd.merge(test_feat, w2v_dic["product_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_pdct = pd.merge(test_feat, w2v_dic["product_category"], how="left", on=UID).values.reshape(-1, 1, 300)
test_inds = pd.merge(test_feat, w2v_dic["industry"], how="left", on=UID).values.reshape(-1, 1, 300)

In [40]:
# doc2vec

In [41]:
d2v_dic = dict()
for cur_id in ID_SET:
    for fname in feat_fname:
        if fname.startswith(f"d2v_avg_{cur_id}"):
            print("current filename: ", fname)
            d2v_dic[cur_id] = pd.read_pickle(f"{UFEDIR}/{fname}")

current filename:  d2v_avg_creative_id.pkl
current filename:  d2v_avg_ad_id.pkl
current filename:  d2v_avg_product_id.pkl
current filename:  d2v_avg_product_category.pkl
current filename:  d2v_avg_advertiser_id.pkl
current filename:  d2v_avg_industry.pkl


In [42]:
train_dctid = pd.merge(train_feat, d2v_dic["creative_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_dadid = pd.merge(train_feat, d2v_dic["ad_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_datid = pd.merge(train_feat, d2v_dic["advertiser_id"], how="left", on=UID).values.reshape(-1, 1, 300)

train_dpdid = pd.merge(train_feat, d2v_dic["product_id"], how="left", on=UID).values.reshape(-1, 1, 300)
train_dpdct = pd.merge(train_feat, d2v_dic["product_category"], how="left", on=UID).values.reshape(-1, 1, 300)
train_dinds = pd.merge(train_feat, d2v_dic["industry"], how="left", on=UID).values.reshape(-1, 1, 300)

In [43]:
test_dctid = pd.merge(test_feat, d2v_dic["creative_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_dadid = pd.merge(test_feat, d2v_dic["ad_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_datid = pd.merge(test_feat, d2v_dic["advertiser_id"], how="left", on=UID).values.reshape(-1, 1, 300)

test_dpdid = pd.merge(test_feat, d2v_dic["product_id"], how="left", on=UID).values.reshape(-1, 1, 300)
test_dpdct = pd.merge(test_feat, d2v_dic["product_category"], how="left", on=UID).values.reshape(-1, 1, 300)
test_dinds = pd.merge(test_feat, d2v_dic["industry"], how="left", on=UID).values.reshape(-1, 1, 300)

In [44]:
# prediction and stats

In [45]:
train_pemb = pd.DataFrame()
test_pemb = pd.DataFrame()
train_pemb[UID] = train_user[UID]
test_pemb[UID] = test_user[UID]

In [46]:
for fname in feat_fname:
    if fname.startswith("meta_"):
        print("current filename: ", fname)
        gender_agg_pred = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_pemb = pd.merge(train_pemb, gender_agg_pred, how="left", on=UID)
        test_pemb = pd.merge(test_pemb, gender_agg_pred, how="left", on=UID)
        gender_agg_pred = None

current filename:  meta_age_group_regeress.pkl
current filename:  meta_gender_group_regeress.pkl


In [47]:
for fname in feat_fname:
    if fname.startswith("tfidf_count_emb_"):
        print("current filename: ", fname)
        cur_tce = pd.read_pickle(f"{UFEDIR}/{fname}")
        train_pemb = pd.merge(train_pemb, cur_tce, how="left", on=UID)
        test_pemb = pd.merge(test_pemb, cur_tce, how="left", on=UID)

current filename:  tfidf_count_emb_age_all.pkl
current filename:  tfidf_count_emb_gender_all.pkl


In [48]:
# stats cate target encode
for fname in feat_fname:
    if fname.startswith("train_stats_catemlb"):
        print("current filename: ", fname)
        train_pemb = pd.merge(train_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_catemlb"):
        print("current filename: ", fname)
        test_pemb = pd.merge(test_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

current filename:  test_stats_catemlb_ad_id.pkl
current filename:  test_stats_catemlb_advertiser_id.pkl
current filename:  test_stats_catemlb_creative_id.pkl
current filename:  test_stats_catemlb_industry.pkl
current filename:  test_stats_catemlb_product_category.pkl
current filename:  test_stats_catemlb_product_id.pkl
current filename:  train_stats_catemlb_ad_id.pkl
current filename:  train_stats_catemlb_advertiser_id.pkl
current filename:  train_stats_catemlb_creative_id.pkl
current filename:  train_stats_catemlb_industry.pkl
current filename:  train_stats_catemlb_product_category.pkl
current filename:  train_stats_catemlb_product_id.pkl


In [49]:
# stats o1
for fname in feat_fname:
    if fname.startswith("train_stats_o1"):
        print("current filename: ", fname)
        train_pemb = pd.merge(train_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_stats_o1"):
        print("current filename: ", fname)
        test_pemb = pd.merge(test_pemb, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

current filename:  test_stats_o1.pkl
current filename:  train_stats_o1.pkl


In [50]:
train_pemb.set_index(UID, inplace=True)
test_pemb.set_index(UID, inplace=True)

In [51]:
set(train_pemb.columns == train_pemb.columns)

{True}

In [52]:
# # stats feature

In [53]:
# train_stat = pd.DataFrame()
# test_stat = pd.DataFrame()
# train_stat[UID] = train_user[UID]
# test_stat[UID] = test_user[UID]

In [54]:
# # stats o1
# for fname in feat_fname:
#     if fname.startswith("train_stats_o1"):
#         print("current filename: ", fname)
#         train_stat = pd.merge(train_stat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
#     elif fname.startswith("test_stats_o1"):
#         print("current filename: ", fname)
#         test_stat = pd.merge(test_stat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [55]:
# # stats getidxmax
# for fname in feat_fname:
#     if fname.startswith("train_stats_o2_getidxmax"):
#         print("current filename: ", fname)
#         train_stat = pd.merge(train_stat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
#     elif fname.startswith("test_stats_o2_getidxmax"):
#         print("current filename: ", fname)
#         test_stat = pd.merge(test_stat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [56]:
# # to make sure feat and user(target) have same order
# # if true --> sum == 0
# np.sum(train_stat.index != train_user.index)

In [57]:
# # to make sure feat and user(target) have same order
# # if true --> sum == 0
# np.sum(test_stat.index != test_user.index)

In [58]:
# # to make sure train and test features have same order
# # if true --> sum == 0
# np.sum(train_stat.columns != test_stat.columns)

In [59]:
# train_stat.set_index(UID, inplace=True)
# test_stat.set_index(UID, inplace=True)

In [60]:
# fillna and inf

In [61]:
def nnaninf(val):
    return np.mean(val[(~np.isnan(val))&(~np.isinf(val))].values)

In [62]:
def fillmean(df):
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            df[col] = df[col].replace([np.nan, np.inf], nnaninf(df[col]))
    # check
    for col in tqdm(df.columns):
        if df[col].count() < len(df):
            log(col)
    
    return df

In [63]:
train_pemb = fillmean(train_pemb)
test_pemb = fillmean(test_pemb)

100%|██████████| 219/219 [00:00<00:00, 528.89it/s]


In [64]:
print(train_pemb.values.max(), train_pemb.values.min())
print(test_pemb.values.max(), test_pemb.values.min())

620639.0 -7.8203125
4789123.0 -7.1171875


In [65]:
# train_stat = fillmean(train_stat)
# test_stat = fillmean(test_stat)

In [66]:
# print(train_stat.values.max(), train_stat.values.min())
# print(test_stat.values.max(), test_stat.values.min())

In [67]:
def transmms(tr_df, te_df):
    tol_df = pd.concat([tr_df, te_df])
    mms = MinMaxScaler()
    mms_tol_df = mms.fit_transform(tol_df)
    mms_tr_df = mms_tol_df[:len(tr_df)]
    mms_te_df = mms_tol_df[len(tr_df):]
    
    return mms_tr_df, mms_te_df

In [68]:
mms_tr_df, mms_te_df = transmms(train_pemb, test_pemb)

In [ ]:
# mms_tr_stat_df, mms_te_stat_df = transmms(train_stat, test_stat)

In [70]:
print(mms_tr_df.max(), mms_tr_df.min())
print(mms_te_df.max(), mms_te_df.min())

1.0000000000000002 0.0
1.0000000000000002 0.0


In [72]:
mms_tr_df.shape

(900000, 219)

In [75]:
mms_te_df.shape

(1000000, 219)

In [71]:
(900000,1,300)

(900000, 1, 300)

In [76]:
### nn k-fold

In [81]:
def sim_lstm_atten(maxlen, num_classes):
    inp_ctid = Input(shape=(1,300))
    inp_adid = Input(shape=(1,300))
    inp_atid = Input(shape=(1,300))

    inp_pdid = Input(shape=(1,300))
    inp_pdct = Input(shape=(1,300))
    inp_inds = Input(shape=(1,300))

    # Stage ctid
    x = inp_ctid
    # x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp_ctid)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)
    # z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2020), activation = "tanh")(y)

    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool_1 = GlobalAveragePooling1D()(y)
    max_pool_1 = GlobalMaxPooling1D()(y)
    # max_pool1 = GlobalMaxPooling1D()(z)

    # convs = []
    # filter_sizes = [2, 3, 5, 8]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn = concatenate(convs, axis=1)

    # Stage adid
    x = inp_adid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_adid)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_3 = Attention(maxlen)(x)
    atten_4 = Attention(maxlen)(y)
    avg_pool_2 = GlobalAveragePooling1D()(y)
    max_pool_2 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_1 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_atid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_5 = Attention(maxlen)(x)
    atten_6 = Attention(maxlen)(y)
    avg_pool_3 = GlobalAveragePooling1D()(y)
    max_pool_3 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_pdid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_7 = Attention(maxlen)(x)
    atten_8 = Attention(maxlen)(y)
    avg_pool_4 = GlobalAveragePooling1D()(y)
    max_pool_4 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_pdct
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_9 = Attention(maxlen)(x)
    atten_10 = Attention(maxlen)(y)
    avg_pool_5 = GlobalAveragePooling1D()(y)
    max_pool_5 = GlobalMaxPooling1D()(y)
    
    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_inds
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_11 = Attention(maxlen)(x)
    atten_12 = Attention(maxlen)(y)
    avg_pool_6 = GlobalAveragePooling1D()(y)
    max_pool_6 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    inp_dctid = Input(shape=(1,300))
    inp_dadid = Input(shape=(1,300))
    inp_datid = Input(shape=(1,300))

    inp_dpdid = Input(shape=(1,300))
    inp_dpdct = Input(shape=(1,300))
    inp_dinds = Input(shape=(1,300))    


    # Stage ctid
    x = inp_dctid
    # x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp_dctid)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)
    # z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2020), activation = "tanh")(y)

    atten_13 = Attention(maxlen)(x) # skip connect
    atten_14 = Attention(maxlen)(y)
    avg_pool_7 = GlobalAveragePooling1D()(y)
    max_pool_7 = GlobalMaxPooling1D()(y)
    # max_pool1 = GlobalMaxPooling1D()(z)

    # convs = []
    # filter_sizes = [2, 3, 5, 8]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn = concatenate(convs, axis=1)

    # Stage adid
    x = inp_dadid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_dadid)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_15 = Attention(maxlen)(x)
    atten_16 = Attention(maxlen)(y)
    avg_pool_8 = GlobalAveragePooling1D()(y)
    max_pool_8 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_1 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_datid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_dusage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_17 = Attention(maxlen)(x)
    atten_18 = Attention(maxlen)(y)
    avg_pool_9 = GlobalAveragePooling1D()(y)
    max_pool_9 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_dpdid
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_dusage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_19 = Attention(maxlen)(x)
    atten_20 = Attention(maxlen)(y)
    avg_pool_10 = GlobalAveragePooling1D()(y)
    max_pool_10 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_dpdct
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_dusage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_21 = Attention(maxlen)(x)
    atten_22 = Attention(maxlen)(y)
    avg_pool_11 = GlobalAveragePooling1D()(y)
    max_pool_11 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)

    # Stage atid
    x = inp_dinds
    # x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_dusage)
    # x = Position_Embedding()(x)
    # x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(GRU(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)

    atten_23 = Attention(maxlen)(x)
    atten_24 = Attention(maxlen)(y)
    avg_pool_12 = GlobalAveragePooling1D()(y)
    max_pool_12 = GlobalMaxPooling1D()(y)

    # convs = []
    # filter_sizes = [2, 4, 6, 10]
    # for fsz in filter_sizes:
    #     l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
    #     l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
    #     l_pool = Flatten()(l_pool)
    #     convs.append(l_pool)
    # text_cnn_2 = concatenate(convs, axis=1)
    
    
    # Stage prediction embedding
    inp_pemb = Input(shape=(219,))
    
    aux_1 = Dense(32)(inp_pemb)
#     aux_1 = BatchNormalization()(x)

    # conc = concatenate([atten_1, atten_2, text_cnn, atten_3, atten_4, text_cnn_1, atten_5, atten_6, text_cnn_2,])
    conc = concatenate([atten_1, atten_2, avg_pool_1, max_pool_1,
                        atten_3, atten_4, avg_pool_2, max_pool_2,
                        atten_5, atten_6, avg_pool_3, max_pool_3,
                        atten_7, atten_8, avg_pool_4, max_pool_4,
                        atten_9, atten_10, avg_pool_5, max_pool_5,
                        atten_11, atten_12, avg_pool_6, max_pool_6,
                        atten_13, atten_14, avg_pool_7, max_pool_7,
                        atten_15, atten_16, avg_pool_8, max_pool_8,
                        atten_17, atten_18, avg_pool_9, max_pool_9,
                        atten_19, atten_20, avg_pool_10, max_pool_10,
                        atten_21, atten_22, avg_pool_11, max_pool_11,
                        atten_23, atten_24, avg_pool_12, max_pool_12,
                        aux_1])
    
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)

    conc = Dense(128)(conc)
    conc = BatchNormalization()(conc)
    # conc = PReLU()(conc)

    outp = Dense(num_classes, activation="softmax")(conc)    

    model = Model(inputs=[inp_ctid, inp_adid, inp_atid, inp_pdid, inp_pdct, inp_inds, 
                         inp_dctid, inp_dadid, inp_datid, inp_dpdid, inp_dpdct, inp_dinds,
                         inp_pemb, ], outputs=outp)

    return model

In [82]:
DATA_SPLIT_SEED = 2020
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]
maxlen=200
# for age
num_classes = 10
train_y = np_utils.to_categorical(train_user["age"]-1)
# # for gender
# num_classes = 2
# train_y = np_utils.to_categorical(train_user["gender"]-1)

test_set = [test_ctid, test_adid, test_atid, test_pdid, test_pdct, test_inds, 
            test_dctid, test_dadid, test_datid, test_dpdid, test_dpdct, test_dinds, 
           mms_te_df, ]
predicted_train_age = np.zeros(train_y.shape)
predicted_age = np.zeros((len(test_user),num_classes))
splits = list(KFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_ctid, train_y))

for idx, (train_idx, valid_idx) in enumerate(splits):
    K.clear_session()

    X_train_ctid = train_ctid[train_idx]
    X_train_adid = train_adid[train_idx]
    X_train_atid = train_atid[train_idx]
    X_train_pdid = train_pdid[train_idx]
    X_train_pdct = train_pdct[train_idx]
    X_train_inds = train_inds[train_idx]
    
    X_train_pemb = mms_tr_df[train_idx]
    
    X_train_dctid = train_dctid[train_idx]
    X_train_dadid = train_dadid[train_idx]
    X_train_datid = train_datid[train_idx]
    X_train_dpdid = train_dpdid[train_idx]
    X_train_dpdct = train_dpdct[train_idx]
    X_train_dinds = train_dinds[train_idx]

    X_val_train_ctid = train_ctid[valid_idx]
    X_val_train_adid = train_adid[valid_idx]
    X_val_train_atid = train_atid[valid_idx]
    X_val_train_pdid = train_pdid[valid_idx]
    X_val_train_pdct = train_pdct[valid_idx]
    X_val_train_inds = train_inds[valid_idx]
    
    X_val_train_pemb = mms_tr_df[valid_idx]

    X_val_train_dctid = train_dctid[valid_idx]
    X_val_train_dadid = train_dadid[valid_idx]
    X_val_train_datid = train_datid[valid_idx]
    X_val_train_dpdid = train_dpdid[valid_idx]
    X_val_train_dpdct = train_dpdct[valid_idx]
    X_val_train_dinds = train_dinds[valid_idx]


    y_train = train_y[train_idx]
    y_val = train_y[valid_idx]

    X_train_set = [X_train_ctid, X_train_adid, X_train_atid, X_train_pdid, X_train_pdct, X_train_inds, 
                  X_train_dctid, X_train_dadid, X_train_datid, X_train_dpdid, X_train_dpdct, X_train_dinds,
                  X_train_pemb, ]
    X_val_set = [X_val_train_ctid, X_val_train_adid, X_val_train_atid, X_val_train_pdid, X_val_train_pdct, X_val_train_inds, 
                X_val_train_dctid, X_val_train_dadid, X_val_train_datid, X_val_train_dpdid, X_val_train_dpdct, X_val_train_dinds, 
                X_val_train_pemb, ]

    model = sim_lstm_atten(maxlen, num_classes)
    model.compile(loss='categorical_crossentropy', optimizer=RAdam(lr=0.001), metrics=['acc'])
    print("MODEL COMPLIE...")
    model.summary()

    model.fit(X_train_set, y_train, batch_size=200, epochs=50, validation_data=(X_val_set, y_val), callbacks=cb, verbose=1)
    pred_val_y = model.predict(X_val_set, batch_size=200, verbose=1)
    pred_test_y = model.predict(test_set, batch_size=200, verbose=1)
    predicted_train_age[valid_idx] = pred_val_y
    predicted_age += pred_test_y / len(splits)
    
accuracy_score(np.argmax(train_y,axis=1),np.argmax(predicted_train_age,axis=1))

MODEL COMPLIE...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1, 300)       0                                            
___________________________________________________________________________


Train on 720000 samples, validate on 180000 samples
Epoch 1/50
720000/720000 [==============================] - 911s 1ms/step - loss: 1.4115 - acc: 0.4274 - val_loss: 1.3288 - val_acc: 0.4507
Epoch 2/50
720000/720000 [==============================] - 881s 1ms/step - loss: 1.3325 - acc: 0.4495 - val_loss: 1.3159 - val_acc: 0.4552
Epoch 3/50
720000/720000 [==============================] - 875s 1ms/step - loss: 1.3142 - acc: 0.4566 - val_loss: 1.3113 - val_acc: 0.4536
Epoch 4/50
720000/720000 [==============================] - 876s 1ms/step - loss: 1.3013 - acc: 0.4612 - val_loss: 1.3032 - val_acc: 0.4599
Epoch 5/50
720000/720000 [==============================] - 864s 1ms/step - loss: 1.2901 - acc: 0.4655 - val_loss: 1.3030 - val_acc: 0.4601
Epoch 6/50
720000/720000 [==============================] - 873s 1ms/step - loss: 1.2791 - acc: 0.4692 - val_loss: 1.3014 - val_acc: 0.4593
Epoch 7/50
720000/720000 [==============================] - 880s 1ms/step - loss: 1.2683 - acc: 0.4739 - va

Train on 720000 samples, validate on 180000 samples
Epoch 1/50
   400/720000 [..............................] - ETA: 11:57:37 - loss: 3.1257 - acc: 0.1075

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.267313). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


720000/720000 [==============================] - 908s 1ms/step - loss: 1.4204 - acc: 0.4256 - val_loss: 1.3424 - val_acc: 0.4453
Epoch 2/50
720000/720000 [==============================] - 890s 1ms/step - loss: 1.3353 - acc: 0.4496 - val_loss: 1.3148 - val_acc: 0.4576
Epoch 3/50
720000/720000 [==============================] - 891s 1ms/step - loss: 1.3172 - acc: 0.4557 - val_loss: 1.3162 - val_acc: 0.4546
Epoch 4/50
720000/720000 [==============================] - 889s 1ms/step - loss: 1.3043 - acc: 0.4600 - val_loss: 1.3061 - val_acc: 0.4569
Epoch 5/50
720000/720000 [==============================] - 891s 1ms/step - loss: 1.2932 - acc: 0.4641 - val_loss: 1.3062 - val_acc: 0.4599
Epoch 6/50
720000/720000 [==============================] - 894s 1ms/step - loss: 1.2830 - acc: 0.4679 - val_loss: 1.3072 - val_acc: 0.4572
Epoch 7/50
720000/720000 [==============================] - 898s 1ms/step - loss: 1.2727 - acc: 0.4721 - val_loss: 1.3029 - val_acc: 0.4616
Epoch 8/50
720000/720000 [=====

Train on 720000 samples, validate on 180000 samples
Epoch 1/50
   400/720000 [..............................] - ETA: 12:14:42 - loss: 3.0505 - acc: 0.0975

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.280202). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


720000/720000 [==============================] - 959s 1ms/step - loss: 1.4214 - acc: 0.4262 - val_loss: 1.3604 - val_acc: 0.4368
Epoch 2/50
720000/720000 [==============================] - 934s 1ms/step - loss: 1.3339 - acc: 0.4495 - val_loss: 1.3215 - val_acc: 0.4510
Epoch 3/50
720000/720000 [==============================] - 937s 1ms/step - loss: 1.3159 - acc: 0.4563 - val_loss: 1.3136 - val_acc: 0.4556
Epoch 4/50
720000/720000 [==============================] - 935s 1ms/step - loss: 1.3036 - acc: 0.4607 - val_loss: 1.3107 - val_acc: 0.4568
Epoch 5/50
720000/720000 [==============================] - 944s 1ms/step - loss: 1.2929 - acc: 0.4646 - val_loss: 1.3078 - val_acc: 0.4566
Epoch 6/50
720000/720000 [==============================] - 949s 1ms/step - loss: 1.2829 - acc: 0.4690 - val_loss: 1.3037 - val_acc: 0.4596
Epoch 7/50
720000/720000 [==============================] - 948s 1ms/step - loss: 1.2732 - acc: 0.4717 - val_loss: 1.3051 - val_acc: 0.4589
Epoch 8/50
720000/720000 [=====

KeyboardInterrupt: 

In [88]:
pd.Series(np.argmax(predicted_age, axis=1)+1).value_counts()

3     253015
2     176461
4     172392
5     131855
6     125634
7      53348
1      26667
8      25059
9      24135
10     11434
dtype: int64

In [93]:
for idx, (train_idx, valid_idx) in enumerate(splits):
    print(accuracy_score(np.argmax(train_y[valid_idx],axis=1),np.argmax(predicted_train_age[valid_idx],axis=1)))

0.4562333333333333
0.45637222222222223
0.4559666666666667
0.03961666666666667
0.039127777777777775


In [110]:
(0.4562333333333333+0.45637222222222223+0.4559666666666667)/3

0.4561907407407408

In [94]:
DATA_SPLIT_SEED = 2020
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]

maxlen=200
# for age
num_classes = 2
train_y = np_utils.to_categorical(train_user["gender"]-1)
# # for gender
# num_classes = 2
# train_y = np_utils.to_categorical(train_user["gender"]-1)

test_set = [test_ctid, test_adid, test_atid, test_pdid, test_pdct, test_inds, 
            test_dctid, test_dadid, test_datid, test_dpdid, test_dpdct, test_dinds, 
           mms_te_df, ]
predicted_train_gender = np.zeros(train_y.shape)
predicted_gender = np.zeros((len(test_user),num_classes))
splits = list(KFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_ctid, train_y))

for idx, (train_idx, valid_idx) in enumerate(splits):
    K.clear_session()

    X_train_ctid = train_ctid[train_idx]
    X_train_adid = train_adid[train_idx]
    X_train_atid = train_atid[train_idx]
    X_train_pdid = train_pdid[train_idx]
    X_train_pdct = train_pdct[train_idx]
    X_train_inds = train_inds[train_idx]
    
    X_train_pemb = mms_tr_df[train_idx]
    
    X_train_dctid = train_dctid[train_idx]
    X_train_dadid = train_dadid[train_idx]
    X_train_datid = train_datid[train_idx]
    X_train_dpdid = train_dpdid[train_idx]
    X_train_dpdct = train_dpdct[train_idx]
    X_train_dinds = train_dinds[train_idx]

    X_val_train_ctid = train_ctid[valid_idx]
    X_val_train_adid = train_adid[valid_idx]
    X_val_train_atid = train_atid[valid_idx]
    X_val_train_pdid = train_pdid[valid_idx]
    X_val_train_pdct = train_pdct[valid_idx]
    X_val_train_inds = train_inds[valid_idx]
    
    X_val_train_pemb = mms_tr_df[valid_idx]

    X_val_train_dctid = train_dctid[valid_idx]
    X_val_train_dadid = train_dadid[valid_idx]
    X_val_train_datid = train_datid[valid_idx]
    X_val_train_dpdid = train_dpdid[valid_idx]
    X_val_train_dpdct = train_dpdct[valid_idx]
    X_val_train_dinds = train_dinds[valid_idx]


    y_train = train_y[train_idx]
    y_val = train_y[valid_idx]

    X_train_set = [X_train_ctid, X_train_adid, X_train_atid, X_train_pdid, X_train_pdct, X_train_inds, 
                  X_train_dctid, X_train_dadid, X_train_datid, X_train_dpdid, X_train_dpdct, X_train_dinds,
                  X_train_pemb, ]
    X_val_set = [X_val_train_ctid, X_val_train_adid, X_val_train_atid, X_val_train_pdid, X_val_train_pdct, X_val_train_inds, 
                X_val_train_dctid, X_val_train_dadid, X_val_train_datid, X_val_train_dpdid, X_val_train_dpdct, X_val_train_dinds, 
                X_val_train_pemb, ]

    model = sim_lstm_atten(maxlen, num_classes)
    model.compile(loss='categorical_crossentropy', optimizer=RAdam(lr=0.001), metrics=['acc'])
    print("MODEL COMPLIE...")
    model.summary()

    model.fit(X_train_set, y_train, batch_size=200, epochs=50, validation_data=(X_val_set, y_val), callbacks=cb, verbose=1)
    pred_val_y = model.predict(X_val_set, batch_size=200, verbose=1)
    pred_test_y = model.predict(test_set, batch_size=200, verbose=1)
    predicted_train_gender[valid_idx] = pred_val_y
    predicted_gender += pred_test_y / len(splits)

accuracy_score(np.argmax(train_y,axis=1),np.argmax(predicted_train_gender,axis=1))

MODEL COMPLIE...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1, 300)       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 1, 300)       0                                            
___________________________________________________________________________

Train on 720000 samples, validate on 180000 samples
Epoch 1/50
   400/720000 [..............................] - ETA: 12:25:53 - loss: 0.8669 - acc: 0.5075

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.278560). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


720000/720000 [==============================] - 983s 1ms/step - loss: 0.1898 - acc: 0.9317 - val_loss: 0.1706 - val_acc: 0.9387
Epoch 2/50
720000/720000 [==============================] - 958s 1ms/step - loss: 0.1743 - acc: 0.9377 - val_loss: 0.1689 - val_acc: 0.9398
Epoch 3/50
720000/720000 [==============================] - 959s 1ms/step - loss: 0.1705 - acc: 0.9390 - val_loss: 0.1676 - val_acc: 0.9405
Epoch 4/50
720000/720000 [==============================] - 967s 1ms/step - loss: 0.1675 - acc: 0.9401 - val_loss: 0.1683 - val_acc: 0.9397
Epoch 5/50
720000/720000 [==============================] - 958s 1ms/step - loss: 0.1647 - acc: 0.9412 - val_loss: 0.1650 - val_acc: 0.9410
Epoch 6/50
720000/720000 [==============================] - 959s 1ms/step - loss: 0.1617 - acc: 0.9422 - val_loss: 0.1649 - val_acc: 0.9414
Epoch 7/50
720000/720000 [==============================] - 964s 1ms/step - loss: 0.1590 - acc: 0.9435 - val_loss: 0.1664 - val_acc: 0.9406
Epoch 8/50
720000/720000 [=====

Train on 720000 samples, validate on 180000 samples
Epoch 1/50
   400/720000 [..............................] - ETA: 12:05:57 - loss: 0.9175 - acc: 0.5500

/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (0.271523). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


720000/720000 [==============================] - 961s 1ms/step - loss: 0.1911 - acc: 0.9314 - val_loss: 0.1730 - val_acc: 0.9382
Epoch 2/50
 74200/720000 [==>...........................] - ETA: 13:11 - loss: 0.1719 - acc: 0.9385

KeyboardInterrupt: 

In [97]:
for idx, (train_idx, valid_idx) in enumerate(splits):
    print(accuracy_score(np.argmax(train_y[valid_idx],axis=1),np.argmax(predicted_train_gender[valid_idx],axis=1)))

0.9398
0.6710166666666667
0.6699944444444444
0.6692166666666667
0.6689833333333334


In [96]:
pd.Series(np.argmax(predicted_gender, axis=1)+1).value_counts()

1    679272
2    320728
dtype: int64

In [113]:
0.9398 + 0.4561907407407408 + 0.015

1.4109907407407407

In [ ]:
# # TODO: 
# 1. add stats feature and prediction embedding
# 2. optimize embedding
# 3. adjust network structure

In [ ]:
0.4299088888888889 + 0.9316455555555555

In [ ]:
## TODO 

1. add stats feature
预期，提高0.003

In [ ]:
2. optimize embedding

In [ ]:
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
with open(f"{UDDIR}/imd/seqidx_dic.pkl", "rb") as f:
    seqidx_dic = pickle.loads(pickle.load(f))

In [ ]:
comb_ids = list(combinations(ID_SET, 2))

In [ ]:
is_eq = True
for cp in comb_ids:
    if (seqidx_dic[cp[0]] != seqidx_dic[cp[1]]).sum() != 0:
        print(cp)
        is_eq = False
        break
assert is_eq, "Must True"

In [ ]:
# reorder index
re_train_user = pd.DataFrame(index=seqidx_dic["creative_id"][seqidx_dic["creative_id"].isin(train_user[UID])])
re_train_user = pd.merge(re_train_user, train_user, how="left", on=UID)

In [ ]:
# reorder index
re_test_user = pd.DataFrame(index=seqidx_dic["creative_id"][~seqidx_dic["creative_id"].isin(train_user[UID])])
re_test_user = pd.merge(re_test_user, test_user, how="left", on=UID)

In [ ]:
train_y = np_utils.to_categorical(re_train_user["age"]-1)
num_classes = 10

In [ ]:
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]

In [ ]:
col = "creative_id"

In [ ]:
train_seq = pd.read_pickle(f"{UDDIR}/imd/train_{col}_seq.pkl")
test_seq = pd.read_pickle(f"{UDDIR}/imd/test_{col}_seq.pkl")

In [ ]:
train_seq_val = train_seq[col].values
test_seq_val = test_seq[col].values

In [ ]:
# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(list(train_seq_val))

In [ ]:
train_X = tokenizer.texts_to_sequences(train_seq_val)
test_X = tokenizer.texts_to_sequences(test_seq_val)

In [ ]:
# Pad the sentences 
train_X = pad_sequences(train_X, maxlen=35)
test_X = pad_sequences(test_X, maxlen=35)

In [ ]:
np.random.choice(train_X, 10)

In [ ]:
word_index = tokenizer.word_index

In [ ]:
input1 = Input(shape=(35,))

In [ ]:
embed_1 = Embedding(35, 300)(input1)

In [ ]:
# #     lstm_1 = LSTM(256)
# dense_1 = Dense(128, kernel_initializer='normal', activation='relu')(embed_1)
# #     dense_1 = Dropout(0.5)(dense_1)
# dense_2 = Dense(64, kernel_initializer='normal', activation='relu')(dense_1)
# #     dense_2 = Dropout(0.5)(dense_2)
# dense_3 = Dense(32, kernel_initializer='normal', activation='relu')(dense_2)
# #     dense_3 = Dropout(0.5)(dense_3)
# out     = Dense(10,kernel_initializer='normal', activation='softmax')(dense_3)
# Compile model
# model = Model(inputs=input1, outputs = input1)
# model.compile(loss ='categorical_crossentropy', optimizer="adadelta", metrics=['accuracy'])

In [ ]:
# model.fit(
#     train_X, 
#     train_y, 
#     batch_size=2048, 
#     epochs=10, 
# #     validation_data=([train_X], train_y), 
#     callbacks = cb, 
#     verbose=1
# )

In [ ]:
max_features = 1024
model = Sequential()
model.add(Embedding(2481135, output_dim=256))
model.add(LSTM(128))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
np.max(train_X)

In [ ]:
model = Sequential()
model.add(Embedding(2481116, 64))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be
# no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

In [ ]:
model.compile(loss ='categorical_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [ ]:
model.fit(train_X[:1000], train_y[:1000], batch_size=2048, epochs=10)

In [ ]:
output_array = model.predict(train_X[:10])

In [ ]:

input_array = np.random.randint(1000, size=(32, 10))

assert output_array.shape == (32, 10, 64)

In [ ]:
model.fit(train_seq[col].values[:10000], train_seq[col].values[:10000], batch_size=2048, epochs=10)

In [ ]:
res_softmax= model.predict(test_X[:10000], verbose=1)

In [ ]:
pd.Series(np.argmax(res_softmax, axis=1)+1).value_counts()

In [ ]:
mms = MinMaxScaler()
mms_tol_feat = mms.fit_transform(tol_feat)

In [ ]:
mms_tr_feat = mms_tol_feat[:len(train_feat)]
mms_te_feat = mms_tol_feat[len(train_feat):]

In [ ]:
# every category
# pd.Series(np.argmax(train_y, axis=1)+1).value_counts()

In [ ]:
# split data
st_X_train, st_X_val, st_y_train, st_y_val = train_test_split(mms_tr_feat, train_y, test_size=0.2, random_state=2020)

In [ ]:
model = baseline_model(inshape=(st_X_train.shape[1],), outshape=10)

In [ ]:
model.fit(
    st_X_train, 
    st_y_train, 
    batch_size=2048, 
    epochs=10, 
    validation_data=([st_X_val], st_y_val), 
    callbacks = cb, 
    verbose=1
)

In [ ]:
model.evaluate(st_X_val, st_y_val)

In [ ]:
res_softmax= model.predict(test_feat, verbose=1)

In [ ]:
predicted_age = np.argmax(res_softmax, axis=1)+1

In [ ]:
tmp1 = model.predict(st_X_val, verbose=1)

### LSTM-Attention

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 9000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 300 # max number of words in a question to use

In [ ]:
col = "creative_id"

In [ ]:
train_X = pd.DataFrame()
test_X = pd.DataFrame()
embedding_matrix = np.array([])

In [ ]:
for col in ["creative_id"]:
    cur_train_X, cur_test_X, word_index = loadnprec(col, maxlen=maxlen)
    cur_embedding_matrix = load_glove(col, word_index, max_features)
    # concat
    if len(train_X):
        train_X = np.concatenate((train_X, cur_train_X), axis=1)
    else:
        train_X = cur_train_X
    # concat
    if len(test_X):
        test_X = np.concatenate((test_X, cur_test_X), axis=1)
    else:
        test_X = cur_test_X
    # concat
    if len(embedding_matrix):
        embedding_matrix = np.concatenate((embedding_matrix, cur_embedding_matrix), axis=1)
    else:
        embedding_matrix = cur_embedding_matrix

In [ ]:
embedding_matrix.shape

In [ ]:
train_X = copy.deepcopy(X_train)

In [ ]:
test_X = copy.deepcopy(X_test)

In [ ]:
help(Embedding)

In [ ]:
def model_lstm_atten(max_features, embedding_matrix):
    
    inp_active = Input(shape=(maxlen,))
    
    # Stage ctid
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp_active)
    x = Position_Embedding()(x)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)
    y = Bidirectional(GRU(64, return_sequences=True))(x)
    z = Conv1D(64, kernel_size = 1, kernel_initializer=initializers.he_uniform(seed=2020), activation = "tanh")(y)
    
    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)
    max_pool1 = GlobalMaxPooling1D()(z)
    
    convs = []
    filter_sizes = [2, 3, 5, 8]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    text_cnn = concatenate(convs, axis=1)

    
    # Stage adid
    x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    x = Position_Embedding()(x)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    
    atten_3 = Attention(maxlen)(x)
    atten_4 = Attention(maxlen)(y)
    
    convs = []
    filter_sizes = [2, 4, 6, 10]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    text_cnn_1 = concatenate(convs, axis=1)
    
    # Stage atid
    x = Embedding(u_max_features, embed_size, weights=[u_embedding_matrix], trainable=False)(inp_usage)
    x = Position_Embedding()(x)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNGRU(128, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
    
    atten_3 = Attention(maxlen)(x)
    atten_4 = Attention(maxlen)(y)
    
    convs = []
    filter_sizes = [2, 4, 6, 10]
    for fsz in filter_sizes:
        l_conv = Conv1D(filters=maxlen, kernel_size=fsz, activation='relu')(y)
        l_pool = MaxPooling1D(maxlen - fsz + 1)(l_conv)
        l_pool = Flatten()(l_pool)
        convs.append(l_pool)
    text_cnn_1 = concatenate(convs, axis=1)    
    

      
    
#     x = Dense(32)(aux)
#     x = BatchNormalization()(x)
    
    conc = concatenate([atten_1, atten_2, avg_pool, max_pool, max_pool1, text_cnn,])
    conc = Dense(256)(conc)
    conc = BatchNormalization()(conc)
    conc = PReLU()(conc)
    conc = Dropout(0.2)(conc)
    
    conc = Dense(128)(conc)
    conc = BatchNormalization()(conc)
#     conc = PReLU()(conc)

    outp = Dense(num_classes, activation="softmax")(conc)    

    model = Model(inputs=[inp_active], outputs=outp)
    
    return model

In [ ]:
inshape = (train_X.shape[1],)

In [ ]:
DATA_SPLIT_SEED = 2020
cb = [EarlyStopping(monitor='val_loss', patience=3, verbose=0),
      CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)]

train_meta = np.zeros(train_y.shape)
test_meta = np.zeros((test_X.shape[0],num_classes))
splits = list(KFold(n_splits=5, shuffle=True, random_state=DATA_SPLIT_SEED).split(train_X, train_y))
for idx, (train_idx, valid_idx) in enumerate(splits):
    K.clear_session()
    X_train = train_X[train_idx]
    X_val = train_X[valid_idx]
    y_train = train_y[train_idx]
    y_val = train_y[valid_idx]
    
#     Aux_train = aux_train[train_idx]|
#     Aux_val = aux_train[valid_idx]
    
    model = model_lstm_atten(max_features, embedding_matrix)
#     model = multi_gpu_model(model, gpus=8)
    model.compile(loss='categorical_crossentropy', optimizer=RAdam(lr=0.001), metrics=['acc'])
    print("MODEL COMPLIE...")
#     lookahead = Lookahead(k=5, alpha=0.5) # Initialize Lookahead
#     lookahead.inject(model) # add into model
#     print("LOOKAHEAD INJECT...")
    model.summary()
    model.fit([X_train], y_train, batch_size=1024, epochs=1, validation_data=([X_val], y_val), callbacks = cb, verbose=1)
    pred_val_y = model.predict([X_val], batch_size=2048, verbose=1)
    pred_test_y = model.predict([test_X], batch_size=2048, verbose=1)
    train_meta[valid_idx] = pred_val_y
    test_meta += pred_test_y / len(splits)

print(accuracy_score(np.argmax(train_y,axis=1),np.argmax(train_meta,axis=1)))

In [ ]:
embedding_matrix.shape

In [ ]:
embedding_matrix

In [ ]:
help(K.name_scope)

In [ ]:
help(K.variable)

# Generate Prediction Result

In [ ]:
# model_lgb_multi_age = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_age_20200511045531.model")

In [ ]:
# model_lgb_multi_gender = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_gender_20200511034408.model")

In [101]:
res = test_user[[UID]]

In [102]:
res["predicted_age"] = np.argmax(predicted_age, axis=1)+1
# res["predicted_age"] = predicted_age
# res["predicted_age"] = [list(x).index(max(x))+1 for x in model_lgb_multi_age.predict(test_feat, num_iteration=model_lgb_multi_age.best_iteration)]

In [103]:
res["predicted_gender"] = np.argmax(predicted_gender, axis=1)+1
# res["predicted_gender"] = [list(x).index(max(x))+1 for x in model_lgb_multi_gender.predict(test_feat, num_iteration=model_lgb_multi_gender.best_iteration)]

In [104]:
res_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
res.to_csv(f"{RESDIR}/res-{res_suffix}.csv", index=False)

In [105]:
res["predicted_age"].value_counts()

3     253015
2     176461
4     172392
5     131855
6     125634
7      53348
1      26667
8      25059
9      24135
10     11434
Name: predicted_age, dtype: int64

In [106]:
res["predicted_gender"].value_counts()

1    679272
2    320728
Name: predicted_gender, dtype: int64

In [ ]:
res.shape

In [ ]:
tmp = pd.read_csv(f"{RESDIR}/res-20200528143657.csv")

In [ ]:
tmp["predicted_age"].value_counts()

In [ ]:
tmp["predicted_gender"].value_counts()

In [ ]:
res["predicted_gender"] = tmp["predicted_gender"]

In [ ]:
train_user.gender.value_counts()

In [ ]:
train_user.age.value_counts()

In [ ]:
gc.collect()

In [ ]:
tmp["predicted_age"]

# Cent result to COS

In [ ]:
from ti import session
ti_session = session.Session()

In [ ]:
inputs = ti_session.upload_data(path=f"{RESDIR}/res-20200515004850.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)